In [5]:
import secret
key = secret.key

In [6]:
from MakeDataset import X_train,X_test,y_train,y_test
import numpy as np
import pandas as pd 
from langchain_groq.chat_models import ChatGroq
import os

ACTIVITIES = {
    1: 'WALKING'            ,
    2: 'WALKING_UPSTAIRS'   ,
    3: 'WALKING_DOWNSTAIRS' ,
    4: 'SITTING'            ,
    5: 'STANDING'           ,
    6: 'LAYING'             ,
}

columns = ["accx","accy","accz"]
subject_test = [10,12,13,18,2,20,24,4,9]
subject_train = [1,11,14,15,16,17,19,21,22,23,25,26,27,28,29,3,30,5,6,7,8]

Groq_Token = key

groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}
model = 'llama3.1-70b'

In [7]:
accx_train = X_train[:,:,0]
accy_train = X_train[:,:,1]
accz_train = X_train[:,:,2]

accx_train_1 = accx_train[np.where(y_train==1)]
accy_train_1 = accy_train[np.where(y_train==1)]
accz_train_1 = accz_train[np.where(y_train==1)]

accx_train_2 = accx_train[np.where(y_train==2)]
accy_train_2 = accy_train[np.where(y_train==2)]
accz_train_2 = accz_train[np.where(y_train==2)]

accx_train_3 = accx_train[np.where(y_train==3)]
accy_train_3 = accy_train[np.where(y_train==3)]
accz_train_3 = accz_train[np.where(y_train==3)]

accx_train_4 = accx_train[np.where(y_train==4)]
accy_train_4 = accy_train[np.where(y_train==4)]
accz_train_4 = accz_train[np.where(y_train==4)]

accx_train_5 = accx_train[np.where(y_train==5)]
accy_train_5 = accy_train[np.where(y_train==5)]
accz_train_5 = accz_train[np.where(y_train==5)]

accx_train_6 = accx_train[np.where(y_train==6)]
accy_train_6 = accy_train[np.where(y_train==6)]
accz_train_6 = accz_train[np.where(y_train==6)]




accx_test = X_test[:,:,0]
accy_test = X_test[:,:,1]
accz_test = X_test[:,:,2]

accx_test_1 = accx_test[np.where(y_test==1)]
accy_test_1 = accy_test[np.where(y_test==1)]
accz_test_1 = accz_test[np.where(y_test==1)]

accx_test_2 = accx_test[np.where(y_test==2)]
accy_test_2 = accy_test[np.where(y_test==2)]
accz_test_2 = accz_test[np.where(y_test==2)]

accx_test_3 = accx_test[np.where(y_test==3)]
accy_test_3 = accy_test[np.where(y_test==3)]
accz_test_3 = accz_test[np.where(y_test==3)]

accx_test_4 = accx_test[np.where(y_test==4)]
accy_test_4 = accy_test[np.where(y_test==4)]
accz_test_4 = accz_test[np.where(y_test==4)]

accx_test_5 = accx_test[np.where(y_test==5)]
accy_test_5 = accy_test[np.where(y_test==5)]
accz_test_5 = accz_test[np.where(y_test==5)]

accx_test_6 = accx_test[np.where(y_test==6)]
accy_test_6 = accy_test[np.where(y_test==6)]
accz_test_6 = accz_test[np.where(y_test==6)]

In [8]:
accx_train_1.shape

(21, 500)

# Q1 
## Demonstrate how to use Zero-Shot Learning and Few-Shot Learning to classify human activities based on the featurized accelerometer data. Qualitatively demonstrate the performance of Few-Shot Learning with Zero-Shot Learning. Which method performs better? Why?

Comparing zero shot with few shot.
Zero shot learning aims to recognize the instances from classes that it has never seen before. Generalizing from seen classes to unseen ones purely based on semantic information, which can be quite difficult if the semantic relationships are weak or ambiguous.

Whereas in few shot learning the model has access to a small number of samples from the target class, making it easier to fine-tune or adjust its internal representations for better accuracy.

Out of the following two cells the first one is implemtation of zero shot and the other is of few shot. We can observe that in the first response when the data is directly given for classification the model generalizes the idea of movement in x,y and z directions and using it as premises it classifies the activities. It tries to use the general idea e.g while walking the acceleration in x should be considerably high with respect to the other axis. Similar inferences are made for all the activities.

While in the few shot learning, the model first analyses the data provided for each of the activities and performs some statistical operations and builds the conclusion. Here, the accuracy rate is significantly high as compared to former case. This is beacuse the model now has the acces to relevant data to study the examples and provide better inferences.

So, to put answer in simple words, the zero shot approach doesn't have access to the data and it generalizes its own understandings to the given dataset.On contrary in the few shot approach the model has access to some samples of the data to learn from and then accordingly classify the test data.


In [9]:
# zero shot

query = f"""
You are being provided with experimental data containing acceleration in x, y, and z directions. 
This data is collected from humans performing the following activities:
1. Walking
2. Walking Upstairs
3. Walking Downstairs
4. Sitting
5. Standing
6. Laying

Your task is to analyze the raw data and determine which activity is being represented based on the above-mentioned activities. 
The acceleration data contains multiple readings of the same activity in a 2D array, with the shape (number of samples, number of readings per axis).

Acceleration in x direction: {accx_test_1.tolist()}
Acceleration in y direction: {accy_test_1.tolist()}
Acceleration in z direction: {accz_test_1.tolist()}
Mean in x direction: {accx_test_1.mean()}
Mean in y direction: {accy_test_1.mean()}
Mean in z direction: {accz_test_1.mean()}
Standard deviation in x direction: {accx_test_1.std()}
Standard deviation in y direction: {accy_test_1.std()}
Standard deviation in z direction: {accz_test_1.std()}
Max value in x direction: {accx_test_1.max()}
Max value in y direction: {accy_test_1.max()}
Max value in z direction: {accz_test_1.max()}
Min value in x direction: {accx_test_1.min()}
Min value in y direction: {accy_test_1.min()}
Min value in z direction: {accx_test_1.min()}
Median in x direction: {np.median(accx_test_1)}
Median in y direction: {np.median(accy_test_1)}
Median in z direction: {np.median(accz_test_1)}

Give the answer from the above mentioned activities along with the number specified
"""


 
model_name = model
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print(answer.content)

Based on the given data, the activity being represented is walking.


In [10]:
query = f"""
You are being provided with the experimental data containing the acceleration in x,y and z directions. 
This data is collected by humans performing  following activities and the data is 2 dimensional that means that 
there are multiple data readingsof same activity within the single one provided to you
1.WALKING
Acceleration in x direction: {accx_train_1.ravel()}
Acceleration in y direction: {accy_train_1.ravel()}
Acceleration in z direction: {accz_train_1.ravel()}

2.WALKING_UPSTAIRS
Acceleration in x direction: {accx_train_2.ravel()}
Acceleration in y direction: {accy_train_2.ravel()}
Acceleration in z direction: {accz_train_2.ravel()}

3.WALKING_DOWNSTAIRS
Acceleration in x direction: {accx_train_3.ravel()}
Acceleration in y direction: {accy_train_3.ravel()}
Acceleration in z direction: {accz_train_3.ravel()}

4.SITTING
Acceleration in x direction: {accx_train_4.ravel()}
Acceleration in y direction: {accy_train_4.ravel()}
Acceleration in z direction: {accz_train_4.ravel()}

5.STANDING
Acceleration in x direction: {accx_train_5.ravel()}
Acceleration in y direction: {accy_train_5.ravel()}
Acceleration in z direction: {accz_train_5.ravel()}

6.LAYING
Acceleration in x direction: {accx_train_6.ravel()}
Acceleration in y direction: {accy_train_6.ravel()}
Acceleration in z direction: {accz_train_6.ravel()}


your task is to analyse the data given above and then analyse the following raw data and say what is the data representing out of the above mentioned activities
Your response should not contain any other explanation just answer in format e.g. 2.WALKING_UPSTAIRS

You can perform PCA analysis,statistical analysis like mean,mode,median,standard deviation and other sush relevant analysis to find the typeof activity

Acceleration in x direction: {accx_test_2.ravel()}
Acceleration in y direction: {accy_test_2.ravel()}
Acceleration in z direction: {accz_test_2.ravel()}

""" 

model_name = model
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print(answer.content)


2.WALKING_UPSTAIRS


# Q2
## Quantitatively compare the accuracy of Few-Shot Learning with Decision Trees (You may use a subset of the test set if you encounter rate-limiting issues). Which method performs better? Why?

Inferences :
The LLM is messing up while classifying the the dynamic activities of walking,walking_upstairs and walking downstairs.
Whereas while classifying the static activities it has pretty good accuracy

In [12]:
predictions = []
idxs = []

for i in range(accx_test.shape[0]):
    feed_x = accx_test[i]
    feed_y = accy_test[i]
    feed_z = accz_test[i]
    
    query = f"""
    You are being provided with the experimental data containing the acceleration in x,y and z directions. 
This data is collected by humans performing  following activities and the data is 2 dimensional that means that 
there are multiple data readingsof same activity within the single one provided to you
1.WALKING
Acceleration in x direction: {accx_train_1.ravel()}
Acceleration in y direction: {accy_train_1.ravel()}
Acceleration in z direction: {accz_train_1.ravel()}

2.WALKING_UPSTAIRS
Acceleration in x direction: {accx_train_2.ravel()}
Acceleration in y direction: {accy_train_2.ravel()}
Acceleration in z direction: {accz_train_2.ravel()}

3.WALKING_DOWNSTAIRS
Acceleration in x direction: {accx_train_3.ravel()}
Acceleration in y direction: {accy_train_3.ravel()}
Acceleration in z direction: {accz_train_3.ravel()}

4.SITTING
Acceleration in x direction: {accx_train_4.ravel()}
Acceleration in y direction: {accy_train_4.ravel()}
Acceleration in z direction: {accz_train_4.ravel()}

5.STANDING
Acceleration in x direction: {accx_train_5.ravel()}
Acceleration in y direction: {accy_train_5.ravel()}
Acceleration in z direction: {accz_train_5.ravel()}

6.LAYING
Acceleration in x direction: {accx_train_6.ravel()}
Acceleration in y direction: {accy_train_6.ravel()}
Acceleration in z direction: {accz_train_6.ravel()}


your task is to analyse the data given above and then analyse the following raw data and say what is the data representing out of the above mentioned activities
Your response should not contain any other explanation just answer in format e.g. 2.WALKING_UPSTAIRS

You can perform PCA analysis,statistical analysis like mean,mode,median,standard deviation and other sush relevant analysis to find the typeof activity

Acceleration in x direction: {feed_x.ravel()}
Acceleration in y direction: {feed_y.ravel()}
Acceleration in z direction: {feed_z.ravel()}
    """ 
    
    model_name = model
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
    answer = llm.invoke(query)
    
    print(answer.content)
    words = answer.content.split(".")
    required = words[-1].split(":")[-1]
    predictions.append(words[0])
    idxs.append(i)
    print("Predicted: ",predictions[-1])
    print("Actual: ",y_test[i])
    print()
        


2.WALKING_UPSTAIRS
Predicted:  2
Actual:  3

2.WALKING_UPSTAIRS
Predicted:  2
Actual:  1

2.WALKING_UPSTAIRS
Predicted:  2
Actual:  2

5.STANDING
Predicted:  5
Actual:  5

5.STANDING
Predicted:  5
Actual:  5

2.WALKING_UPSTAIRS
Predicted:  2
Actual:  1

2.WALKING_UPSTAIRS
Predicted:  2
Actual:  1

5.STANDING
Predicted:  5
Actual:  5

2.WALKING_UPSTAIRS
Predicted:  2
Actual:  3

2.WALKING_UPSTAIRS
Predicted:  2
Actual:  2

6.LAYING
Predicted:  6
Actual:  6

5.STANDING
Predicted:  5
Actual:  5

4.SITTING
Predicted:  4
Actual:  6

5.STANDING
Predicted:  5
Actual:  5

6.LAYING
Predicted:  6
Actual:  6

2.WALKING_UPSTAIRS
Predicted:  2
Actual:  1

6.LAYING
Predicted:  6
Actual:  6

2.WALKING_UPSTAIRS
Predicted:  2
Actual:  5

2.WALKING_UPSTAIRS
Predicted:  2
Actual:  2

5.STANDING
Predicted:  5
Actual:  5

5.STANDING
Predicted:  5
Actual:  4

2.WALKING_UPSTAIRS
Predicted:  2
Actual:  3

2.WALKING_UPSTAIRS
Predicted:  2
Actual:  2

2.WALKING_UPSTAIRS
Predicted:  2
Actual:  2

2.WALKING_UPSTA

InternalServerError: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}}

In [14]:
# Checking the predictions data
predictions = [int(i) for i in predictions]
print(predictions)
print(y_test[idxs])
print("\n\n")

y_test_filtered = y_test[idxs]

# Calculate the number of correct predictions
correct_predictions = sum(p == y for p, y in zip(predictions, y_test_filtered))

# Calculate the accuracy
accuracy = correct_predictions / len(predictions)

# Print the accuracy
print(f"Total Accuracy of few shot: {accuracy:.2%}")

dynamic_test = []
dynamic_predict = []

static_test = []
static_predict = []

for i in range(len(y_test_filtered)):
    if y_test_filtered[i] in [1,2,3]:
        dynamic_predict.append(predictions[i])
        dynamic_test.append(y_test_filtered[i])
    
    if y_test_filtered[i] in [4,5,6]:
        static_predict.append(predictions[i])
        static_test.append(y_test_filtered[i])


dynamic_accuracy = sum(p == y for p,y in zip(dynamic_predict,dynamic_test))/len(dynamic_predict)
print(f"Accuracy in Dynamic activities is {dynamic_accuracy:.2%}")

static_accuracy = sum(p == y for p,y in zip(static_predict,static_test))/len(static_predict)
print(f"Accuracy in Static activities is {static_accuracy:.2%}")

[2, 2, 2, 5, 5, 2, 2, 5, 2, 2, 6, 5, 4, 5, 6, 2, 6, 2, 2, 5, 5, 2, 2, 2, 2, 4, 6, 4, 2, 2, 6, 2, 4, 4]
[3 1 2 5 5 1 1 5 3 2 6 5 6 5 6 1 6 5 2 5 4 3 2 2 1 4 6 4 1 2 6 2 4 4]



Total Accuracy of few shot: 64.71%
Accuracy in Dynamic activities is 43.75%
Accuracy in Static activities is 83.33%


# Q3
## What are the limitations of Zero-Shot Learning and Few-Shot Learning in the context of classifying human activities based on featurized accelerometer data?

### Limitations of zero shot and few shot in context of classifying human activities based on featurezied data

1) Semantic Description : ZSL and FSL depends on the quality of prompt we are using to descibe the problem statement.If these prompts are not structured properly then the models ability to recognize the activities may suffer. Somtimes it is difficult to structure the data into semantic manner.
2) Contraints Related to data: At a time we can input only certain quantity of data that to in semantic way. So the model has very less data to work with leading to very high inaccuracy. Also we have to embed the data into semantic prompt which limits our capability to featurize the data and provide all the featurized data.If the data is too noisy then the LLM will struggle to learn from the few samples we provided.
3) Generalization: These LLMs tries to generalize their understanding for the given data. This error can be reduced if we use FSL where we provide some sample data to model for its training. But in ZSL the LLM tries to generalize completely because it has not seen the data we fed before. So even in FSL there is generalization to some extent. 
4) Limited Adaptibility: Both ZSL and FSL face challenges when dealing with activities that are inherently complex or involve multiple stages or types of movements. These models often struggle to accurately classify such activities. In the case of FSL, when an activity not represented in the training data is encountered, the model tends to forcefully classify it as one of the known activities rather than recognizing it as an entirely new or different activity.

# Q4
## What does the model classify when given input from an entirely new activity that it hasn't seen before?

In this scenario, the model has been trained on data from only four specific activities, excluding the 'Standing' and 'Walking' activities, although these two activities are included in the list of possible predictions. When the model is given data related to 'Standing' or 'Walking,' it tends to force the data into one of the four activities it has seen during training, often failing to recognize or accurately classify the 'Standing' or 'Walking' activities. As a result, the model consistently misclassifies these activities because it lacks the necessary training data to correctly identify them, despite their presence in the prediction list.

In [ ]:
query = f"""
You are being provided with the experimental data containing the acceleration in x,y and z directions. 
This data is collected by humans performing  following activities and the data is 2 dimensional that means that 
there are multiple data readingsof same activity within the single one provided to you
1.WALKING
Acceleration in x direction: {accx_train_1.ravel()}
Acceleration in y direction: {accy_train_1.ravel()}
Acceleration in z direction: {accz_train_1.ravel()}

2.WALKING_UPSTAIRS
Acceleration in x direction: {accx_train_2.ravel()}
Acceleration in y direction: {accy_train_2.ravel()}
Acceleration in z direction: {accz_train_2.ravel()}

3.WALKING_DOWNSTAIRS

4.SITTING
Acceleration in x direction: {accx_train_4.ravel()}
Acceleration in y direction: {accy_train_4.ravel()}
Acceleration in z direction: {accz_train_4.ravel()}

5.STANDING

6.LAYING
Acceleration in x direction: {accx_train_6.ravel()}
Acceleration in y direction: {accy_train_6.ravel()}
Acceleration in z direction: {accz_train_6.ravel()}


your task is to analyse the data given above and then analyse the following raw data and say what is the data representing out of the above mentioned activities
Acceleration in x direction: {accx_test_5[5]}
Acceleration in y direction: {accy_test_5[5]}
Acceleration in z direction: {accz_test_5[5]}

""" 

model_name = model
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print(answer.content)


Analyzing the provided data, I will attempt to identify the activity represented by the raw data.

**Observations:**

1. The acceleration values in the x, y, and z directions are relatively stable, with some fluctuations.
2. The x-axis acceleration values are mostly positive, indicating movement in the forward direction.
3. The y-axis acceleration values are mostly negative, indicating movement in the downward direction.
4. The z-axis acceleration values are relatively small, indicating minimal movement in the vertical direction.

**Comparison with the provided activities:**

1. **WALKING**: The x-axis acceleration values are similar to those in the provided walking data, with a range of approximately 0.8 to 1.2. The y-axis acceleration values are also similar, with a range of approximately -0.2 to -0.1.
2. **WALKING_UPSTAIRS**: The x-axis acceleration values are higher than those in the provided walking data, with a range of approximately 1.2 to 1.5. The y-axis acceleration values are

# Q5
## Test the model with random data (ensuring the data has the same dimensions and range as the previous input) and report the results. 

In [ ]:
previous_input = accx_train_5
dim = previous_input.shape
randomx = np.random.rand(dim[0],500)
randomy = np.random.rand(dim[0],500)
randomz = np.random.rand(dim[0],500)
print(randomx.shape)

(21, 500)


In [ ]:
# zero shot

query = f"""
You are being provided with the experimental data containing the acceleration in x,y and z directions. 
This data is collected by humans performing  following activities 
1.Walking
2.Walking Upstairs
3.Walking Downstairs
4.Sitting
5.Standing
6.Laying

your task is to analyse the raw data and say what is the data representing out of the above mentioned activities

Acceleration in x direction: {randomx[0]}
Acceleration in y direction: {randomy[0]}
Acceleration in z direction: {randomz[0]}
""" 

 
model_name = model
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print(answer.content)

Analyzing the provided acceleration data in x, y, and z directions, we can observe the following patterns and characteristics:

1. **Range and Distribution**: The acceleration values in all three directions (x, y, z) range from approximately -1 to 1, with most values concentrated between -0.5 and 0.5. This suggests that the activities being performed do not involve extreme accelerations.

2. **Patterns in X Direction**: The acceleration in the x direction appears to have a mix of high and low values, with some periods of relatively consistent values (e.g., around indices 100-150). This could indicate activities that involve walking or moving in a straight line.

3. **Patterns in Y Direction**: The acceleration in the y direction shows more variability, with frequent changes in magnitude and direction. This could be indicative of activities that involve movement in multiple directions or changes in elevation (e.g., walking upstairs or downstairs).

4. **Patterns in Z Direction**: The ac

In [ ]:
#few shot

query = f"""
You are being provided with the experimental data containing the acceleration in x,y and z directions. 
This data is collected by humans performing  following activities and the data is 2 dimensional that means that 
there are multiple data readingsof same activity within the single one provided to you
1.WALKING
Acceleration in x direction: {accx_train_1.ravel()}
Acceleration in y direction: {accy_train_1.ravel()}
Acceleration in z direction: {accz_train_1.ravel()}

2.WALKING_UPSTAIRS
Acceleration in x direction: {accx_train_2.ravel()}
Acceleration in y direction: {accy_train_2.ravel()}
Acceleration in z direction: {accz_train_2.ravel()}

3.WALKING_DOWNSTAIRS
Acceleration in x direction: {accx_train_3.ravel()}
Acceleration in y direction: {accy_train_3.ravel()}
Acceleration in z direction: {accz_train_3.ravel()}

4.SITTING
Acceleration in x direction: {accx_train_4.ravel()}
Acceleration in y direction: {accy_train_4.ravel()}
Acceleration in z direction: {accz_train_4.ravel()}

5.STANDING
Acceleration in x direction: {accx_train_5.ravel()}
Acceleration in y direction: {accy_train_5.ravel()}
Acceleration in z direction: {accz_train_5.ravel()}

6.LAYING
Acceleration in x direction: {accx_train_6.ravel()}
Acceleration in y direction: {accy_train_6.ravel()}
Acceleration in z direction: {accz_train_6.ravel()}



your task is to analyse the data given above and then analyse the following raw data and say what is the data representing out of the above mentioned activities
Acceleration in x direction: {randomx[4]}
Acceleration in y direction: {randomy[4]}
Acceleration in z direction: {randomz[4]}

""" 

model_name = model
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print(answer.content)


Analyzing the provided data, I will attempt to identify the activity represented by the raw data.

**Observations:**

1. The acceleration values in all three directions (x, y, z) are within the range of -1 to 1, which is consistent with the data provided for the six activities.
2. The x-axis acceleration values are mostly positive, with some negative values, indicating movement in both forward and backward directions.
3. The y-axis acceleration values are mostly positive, with some negative values, indicating movement in both upward and downward directions.
4. The z-axis acceleration values are mostly positive, indicating movement in the upward direction.

**Comparison with the provided data:**

1. **Walking**: The x-axis acceleration values are mostly positive, with some negative values, which is consistent with the walking data. However, the y-axis acceleration values are more positive than the walking data, indicating more upward movement.
2. **Walking Upstairs**: The x-axis acceler